In [1]:
try:
    from google.colab import drive
    drive.mount('/content/drive')   
    RUNNING_IN_COLAB = True
    
except:
    print("Not in Colab")
    RUNNING_IN_COLAB = False
    
if RUNNING_IN_COLAB:
    print("Running in Colab")
    #pull git repo and install requirements
    !git clone https://github.com/tmengel/Deep-Learning.git
    %cd Deep-Learning/finalProject
    %pip install uproot
    %pip install pandas
    %pip install pickle


import pulsenet as pn
import pandas as pd  
import tensorflow as tf
from tensorflow import keras
from keras import layers, models
import pickle as pkl
import numpy as np
from sklearn.metrics import roc_curve

Not in Colab


2023-05-14 15:01:17.491381: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-14 15:01:19.027171: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/tmengel/root/lib
2023-05-14 15:01:19.027436: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-14 15:01:20.652668: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared obj

In [ ]:
filename = "ysoTracesWithPileup.root"
if RUNNING_IN_COLAB:
    filename = "/content/drive/MyDrive/DeepLearningFinalProject/ysoTracesWithPileup.root"

def GetData(filename, pileup_split=0.5, phase_min=0.1, phase_max=20, amplitude_min=0.5, amplitude_max=1.5):
    x_trace, y_trace, y_phase, y_amp = pn.CreateData(filename, pileup_split=pileup_split, phase_min=phase_min, phase_max=phase_max, amplitude_min=amplitude_min, amplitude_max=amplitude_max)
    y_pileup = pn.EncodePileup(y_phase)
    return x_trace, y_trace, y_phase, y_amp, y_pileup

def GetResults(model, phaseMin,phaseMax, filename):
    x_trace, y_trace, y_phase, y_amp, y_pileup = GetData(filename=filename, pileup_split=0.5, phase_min=phaseMin, phase_max=phaseMax, amplitude_min=0.5, amplitude_max=1.5)
    pred_trace, pred_phase, pred_amp, pred_pileup = model.predict(x_trace)
    n_samples = x_trace.shape[0]
    # separate out the traces
    pred_trace1 = pred_trace[:,0,:].reshape(-1, 300)
    pred_trace2 = pred_trace[:,1,:].reshape(-1, 300)
    y_trace1 = y_trace[:,0,:].reshape(-1, 300)
    y_trace2 = y_trace[:,1,:].reshape(-1, 300)
    x_trace = x_trace.reshape(-1, 300)
    
    # calculate residuals
    res_trace1 = pred_trace1 - y_trace1
    res_trace2 = pred_trace2 - y_trace2
    res_trace_total = pred_trace - y_trace
    res_phase = pred_phase - y_phase
    res_amp = pred_amp - y_amp
    
    average_res_trace1 = np.mean(res_trace1, axis=0)
    average_res_trace2 = np.mean(res_trace2, axis=0)
    std_res_trace1 = np.std(res_trace1, axis=0)
    std_res_trace2 = np.std(res_trace2, axis=0)
    
    fpr, tpr, thresholds = roc_curve(y_pileup, pred_pileup)
    
    loss, acc = model.evaluate(x_trace, [y_trace, y_phase, y_amp, y_pileup])
    
    result = {
            "loss": loss,
            "acc": acc,
            "n_samples": n_samples,
            "x_trace": x_trace,
            "y_trace": y_trace,
            "y_phase": y_phase,
            "y_amp": y_amp,
            "y_pileup": y_pileup,
            "pred_trace": pred_trace,
            "pred_phase": pred_phase,
            "pred_amp": pred_amp,
            "pred_pileup": pred_pileup,
            "res_trace1": res_trace1,
            "res_trace2": res_trace2,
            "res_trace_total": res_trace_total,
            "res_phase": res_phase,
            "res_amp": res_amp,
            "average_res_trace1": average_res_trace1,
            "average_res_trace2": average_res_trace2,
            "std_res_trace1": std_res_trace1,
            "std_res_trace2": std_res_trace2,
            "fpr": fpr, "tpr": tpr, "thresholds": thresholds}
    
    return result
    

In [ ]:
input = layers.Input(shape=(1, 300))  # Returns a placeholder tensor
classifer_feature_vec = pn.TraceClassifierBase(name = "classifier_base")(input)
classifier_output = pn.TraceClassifierHead(name = "classifier")(classifer_feature_vec)
phase_output = pn.TracePhaseRegressor(name="phase")(input)
amplitude_output = pn.TraceAmplitudeRegressor(name ="amplitude")(input)
discriminator_feature_vec = pn.TraceDiscriminatorBase(name = "discriminator_base")(input)
trace_output = pn.TraceDiscriminatorHead(name = "discriminator")(discriminator_feature_vec)


model = models.Model(inputs=input, outputs=[trace_output, phase_output, amplitude_output, classifier_output])
model.compile(optimizer='adam', loss=["mse", "mse", "mse", "bce"], metrics='accuracy')
model.build(input_shape=(None, 1, 300))
model_names = [model.layers[i].name for i in range(len(model.layers))]
model.layers[model_names.index("discriminator_base")].load_weights('weights/individual_discriminator_base_weights.h5')
model.layers[model_names.index("discriminator")].load_weights('weights/individual_discriminator_head_weights.h5')
model.layers[model_names.index("classifier_base")].load_weights('weights/individual_classifier_base_weights.h5')
model.layers[model_names.index("classifier")].load_weights('weights/individual_classifier_head_weights.h5')
model.layers[model_names.index("phase")].load_weights('weights/individual_phase_weights_contingent.h5')
model.layers[model_names.index("amplitude")].load_weights('weights/individual_amplitude_weights_contingent.h5')
model.save('models/TraceNet.h5')

!cp -rf models/TraceNet.h5 /content/drive/MyDrive/DeepLearningFinalProject/

In [5]:
phase_shift_bins = [0.1, 5.0, 10.0, 20.0] # 0 , 10, 20, 40 ns
results = {}
results[f'phase_shift_{int(phase_shift_bins[0]*2)}to{int(phase_shift_bins[-1]*2)}ns'] = GetResults(model, phaseMin=phase_shift_bins[0], phaseMax=phase_shift_bins[-1], filename=filename)
for i in range(len(phase_shift_bins)-1):
    print(f'phase_shift_{int(phase_shift_bins[i]*2)}to{int(phase_shift_bins[i+1]*2)}ns')
    results[f'phase_shift_{int(phase_shift_bins[i]*2)}to{int(phase_shift_bins[i+1]*2)}ns'] = GetResults(model, phaseMin=phase_shift_bins[i], phaseMax=phase_shift_bins[i+1], filename=filename)


with open('results.pickle', 'wb') as handle:
    pkl.dump(results, handle, protocol=pkl.HIGHEST_PROTOCOL)
    
!cp -rf results.pickle /content/drive/MyDrive/DeepLearningFinalProject/

20.0
